In [79]:
#load csv
import pandas as pd
pred_personalities = pd.read_csv("predicted_personality_types.csv")

In [80]:
busids = pd.read_json('Business_ids.json',lines=True,orient='index')
biz_ids = pd.DataFrame(busids)
columns = ['business_id']
biz_ids.columns = columns

In [84]:
biz_ids

,business_id
0,dn_ipqbm7_jUz5X3rDez_A
1,LFs5jyYdXlzi0SpAYi1eSA
2,9SU7ZZhaFUJJ6m2k5HKHeg
3,2sx52lDoiEtef7xgPCaoBw
4,BLIJ-p5wYuAhw6Pp6mh6mw
5,bpRo8L8dkhgbJhdIKa9mwA
6,uW6UHfONAmm8QttPkbMewQ
7,CVKOPzBVOj3_apFUmZ9ZWw
8,f9sU31meK0bqAD7922sCog
9,hIUKufhwR6Ifn7bi0-phLA


In [83]:
import feather
reviews1 = feather.read_dataframe('reviews1.feather')
reviews2 = feather.read_dataframe('reviews2.feather')                    

In [85]:
#link predicted personalities to the businesses they commented on 
#1. link user ID to business ID from yelp_review_dataset --> result: BizID_UsrID 
BizID_UsrID1 = pd.DataFrame(reviews1[reviews1['business_id'].isin(biz_ids['business_id'])])
BizID_UsrID2 = pd.DataFrame(reviews2[reviews2['business_id'].isin(biz_ids['business_id'])])
BizID_UsrID1 = BizID_UsrID1.append(BizID_UsrID2)
#BizID_UsrID = BizID_UsrID1.filter(['business_id','user_id'], axis=1)
BizID_UsrID = BizID_UsrID1[['business_id', 'user_id', 'stars']].copy()

In [86]:
#2. Filter records for businesses available in biz_ids
BizID_UsrID = pd.DataFrame(BizID_UsrID[BizID_UsrID['business_id'].isin(biz_ids['business_id'])])

In [87]:
#3. join BizID_UsrID with pred_personalities 
BID_UID_Pers = BizID_UsrID.merge(pred_personalities, on ='user_id')
BID_UID_Pers = BID_UID_Pers[['business_id','user_id','text','EI','FT','NS','PJ', 'funny', 'cool', 'useful', 'stars']].copy()
#Export to CSV 
BID_UID_Pers.to_csv('../Data.bizID_usrID_personality.csv')

In [88]:
#4. aggregate personality types per business --> result BizID_Personality

#Personality types count  
#'I': '1', 'E': '0' / 'T': '1', 'F': '0' / 'N': '1', 'S': '0'/ 'P': '1', 'J': '0'
BID_I = pd.DataFrame(BID_UID_Pers.groupby('business_id')['EI'].sum())
BID_I.rename(columns={'EI':'I'},inplace=True)
BID_E = pd.DataFrame((BID_UID_Pers.groupby('business_id')['EI'].count() - BID_UID_Pers.groupby('business_id')['EI'].sum()))
BID_E.rename(columns={'EI':'E'},inplace=True)
BID_T = pd.DataFrame(BID_UID_Pers.groupby('business_id')['FT'].sum())
BID_T.rename(columns={'FT':'T'},inplace=True)
BID_F = pd.DataFrame((BID_UID_Pers.groupby('business_id')['FT'].count() - BID_UID_Pers.groupby('business_id')['FT'].sum()))
BID_F.rename(columns={'FT':'F'},inplace=True)
BID_N = pd.DataFrame(BID_UID_Pers.groupby('business_id')['NS'].sum())
BID_N.rename(columns={'NS':'N'},inplace=True)
BID_S = pd.DataFrame((BID_UID_Pers.groupby('business_id')['NS'].count() - BID_UID_Pers.groupby('business_id')['NS'].sum()))
BID_S.rename(columns={'NS':'S'},inplace=True)
BID_P = pd.DataFrame(BID_UID_Pers.groupby('business_id')['PJ'].sum())
BID_P.rename(columns={'PJ':'P'},inplace=True)
BID_J = pd.DataFrame((BID_UID_Pers.groupby('business_id')['PJ'].count() - BID_UID_Pers.groupby('business_id')['PJ'].sum()))
BID_J.rename(columns={'PJ':'J'},inplace=True)


In [89]:
#funny, cool, useful, stars
BID_Funny = pd.DataFrame(BID_UID_Pers.groupby('business_id')['funny'].sum())
BID_Cool = pd.DataFrame(BID_UID_Pers.groupby('business_id')['cool'].sum())
BID_Useful = pd.DataFrame(BID_UID_Pers.groupby('business_id')['useful'].sum())
BID_S1 = pd.DataFrame(BID_UID_Pers[BID_UID_Pers['stars'] == 1].groupby('business_id')['user_id'].count())
BID_S1.columns = ['1star']
BID_S2 = pd.DataFrame(BID_UID_Pers[BID_UID_Pers['stars'] == 2].groupby('business_id')['stars'].count())
BID_S2.columns = ['2star']
BID_S3 = pd.DataFrame(BID_UID_Pers[BID_UID_Pers['stars'] == 3].groupby('business_id')['stars'].count())
BID_S3.columns = ['3star']
BID_S4 = pd.DataFrame(BID_UID_Pers[BID_UID_Pers['stars'] == 4].groupby('business_id')['stars'].count())
BID_S4.columns = ['4star']
BID_S5 = pd.DataFrame(BID_UID_Pers[BID_UID_Pers['stars'] == 5].groupby('business_id')['stars'].count())
BID_S5.columns = ['5star']
BID_Total = pd.DataFrame(BID_UID_Pers.groupby('business_id')['stars'].count())
BID_Total.columns = ['total']

In [90]:
# include results in one dataframe
BizID_Personality = BID_I.merge(BID_E, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_T, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_F, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_N, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_S, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_P, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_J, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_Funny, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_Cool, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_Useful, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_S1, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_S2, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_S3, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_S4, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_S5, copy = False, on = 'business_id')
BizID_Personality = BizID_Personality.merge(BID_Total, copy = False, on = 'business_id')


In [97]:
#average personality/ratings

def sum_by_column(new_col_name, list_of_cols_to_sum):
    BizID_Personality[new_col_name] = BizID_Personality[list_of_cols_to_sum].astype(int).sum(axis=1)

def avg_personality(new_col_name, personality_column):
    BizID_Personality[new_col_name] = BizID_Personality[personality_column]/BizID_Personality['PERS_TOTAL']

def add_std(new_col_name, list_of_cols_to_aggregate, op_type):
    BizID_Personality[new_col_name] = BizID_Personality[list_of_cols_to_aggregate].astype(op_type).std(axis=1)

def add_avg(new_col_name, num, den):
    BizID_Personality[new_col_name] = BizID_Personality[num]/BizID_Personality[den] 


    
sum_by_column('PERS_TOTAL', ['I', 'E', 'T', 'F', 'N', 'S', 'P', 'J'])
avg_personality('I_AVG', 'I')
avg_personality('E_AVG', 'E')
avg_personality('T_AVG', 'T')
avg_personality('F_AVG', 'F')
avg_personality('N_AVG', 'N')
avg_personality('S_AVG', 'S')
avg_personality('P_AVG', 'P')
avg_personality('J_AVG', 'J')

add_std('std_personalities', ['I', 'E', 'T', 'F', 'N', 'S', 'P', 'J'], 'float')

add_std('std_stars', ['1star','2star','3star','4star','5star'], 'float')

add_avg('I_5S', 'I', '5star')
add_avg('E_5S', 'E', '5star')
add_avg('T_5S', 'T', '5star')
add_avg('F_5S', 'F', '5star')
add_avg('N_5S', 'N', '5star')
add_avg('S_5S', 'S', '5star')
add_avg('P_5S', 'P', '5star')

add_avg('I_4S', 'I', '4star')
add_avg('E_4S', 'E', '4star')
add_avg('T_4S', 'T', '4star')
add_avg('F_4S', 'F', '4star')
add_avg('N_4S', 'N', '4star')
add_avg('S_4S', 'S', '4star')
add_avg('P_4S', 'P', '4star')

add_avg('I_3S', 'I', '3star')
add_avg('E_3S', 'E', '3star')
add_avg('T_3S', 'T', '3star')
add_avg('F_3S', 'F', '3star')
add_avg('N_3S', 'N', '3star')
add_avg('S_3S', 'S', '3star')
add_avg('P_3S', 'P', '3star')

add_avg('I_2S', 'I', '2star')
add_avg('E_2S', 'E', '2star')
add_avg('T_2S', 'T', '2star')
add_avg('F_2S', 'F', '2star')
add_avg('N_2S', 'N', '2star')
add_avg('S_2S', 'S', '2star')
add_avg('P_2S', 'P', '2star')

add_avg('I_1S', 'I', '1star')
add_avg('E_1S', 'E', '1star')
add_avg('T_1S', 'T', '1star')
add_avg('F_1S', 'F', '1star')
add_avg('N_1S', 'N', '1star')
add_avg('S_1S', 'S', '1star')
add_avg('P_1S', 'P', '1star')

#total personalities per business 
#PERS_TOTAL = pd.DataFrame(BizID_Personality['BID_T'] + BizID_Personality['BID_F'] + BizID_Personality['BID_N'] + BizID_Personality['BID_S'] + BizID_Personality['BID_P'] + BizID_Personality['BID_J'])

#std deviation 

#Export to CSV


In [98]:
BizID_Personality.head

<bound method NDFrame.head of                            I     E    T     F     N     S     P    J  funny  \
business_id                                                                   
-ed0Yc9on37RoIoG2ZgxBA   780   810  115  1475   401  1189  1331  259    819   
2sx52lDoiEtef7xgPCaoBw   615   600   64  1151   314   901  1053  162    296   
3GEEy7RP6e4bT4LAiWFMFQ   945  1017  141  1821   520  1442  1646  316    992   
3uC7Lbc3RgUDTWQlBu4PqQ   536   579   92  1023   295   820   945  170    549   
5shgJB7a-2_gdnzc0gsOtg   621   686  120  1187   374   933  1108  199    854   
6tSvz_21BMo3a4GaItwa0g   532   564   80  1016   278   818   911  185    603   
7ibuDLfx8gEhESAufYIySw   471   540   83   928   266   745   849  162    510   
9SU7ZZhaFUJJ6m2k5HKHeg   961   919  146  1734   449  1431  1637  243    591   
A5Rkh7UymKm0_Rxm9K2PJw   610   692  134  1168   381   921  1028  274   1284   
BLIJ-p5wYuAhw6Pp6mh6mw  1336  1483  236  2583   679  2140  2441  378   1193   
CVKOPzBVOj3_apFUmZ9ZWw

In [99]:
BizID_Personality.to_csv('../Data.bizID_personality_counts.csv')

In [100]:
#5. concat comments per business ID --> result BizID_Comments
BizID_All_Comments = BID_UID_Pers[['business_id','text']].copy()
#join comments per business
BizID_All_Comments = pd.DataFrame(BizID_All_Comments.groupby('business_id')['text'].apply(' '.join))
#export to CSV
BizID_All_Comments.to_csv('../Data.bizID_all_comments.csv')

In [101]:
BizID_Personality.iloc[0]

I                     780.000000
E                     810.000000
T                     115.000000
F                    1475.000000
N                     401.000000
S                    1189.000000
P                    1331.000000
J                     259.000000
funny                 819.000000
cool                 5089.000000
useful                 84.000000
1star                 124.000000
2star                  64.000000
3star                 161.000000
4star                 441.000000
5star                 800.000000
total                1590.000000
PERS_TOTAL           6360.000000
I_PERC                 12.264151
E_PERC                 12.735849
T_PERC                  1.808176
F_PERC                 23.191824
N_PERC                  6.305031
S_PERC                 18.694969
P_PERC                 20.927673
J_PERC                  4.072327
std_personalities     508.543298
std_stars             305.881513
I_5S                    0.975000
E_5S                    1.012500
          

In [102]:
BizID_Personality.count()

I                    56
E                    56
T                    56
F                    56
N                    56
S                    56
P                    56
J                    56
funny                56
cool                 56
useful               56
1star                56
2star                56
3star                56
4star                56
5star                56
total                56
PERS_TOTAL           56
I_PERC               56
E_PERC               56
T_PERC               56
F_PERC               56
N_PERC               56
S_PERC               56
P_PERC               56
J_PERC               56
std_personalities    56
std_stars            56
I_5S                 56
E_5S                 56
                     ..
P_4S                 56
I_3S                 56
E_3S                 56
T_3S                 56
F_3S                 56
N_3S                 56
S_3S                 56
P_3S                 56
I_2S                 56
E_2S                 56
T_2S            

In [103]:
BizID_All_Comments.head

<bound method NDFrame.head of                                                                      text
business_id                                                              
-ed0Yc9on37RoIoG2ZgxBA  I love how they have no rice sushi options. Th...
2sx52lDoiEtef7xgPCaoBw  Food is always delicious, definitely my favori...
3GEEy7RP6e4bT4LAiWFMFQ  Atmosphere awesome\r\r\r\nPizza perfection\r\r...
3uC7Lbc3RgUDTWQlBu4PqQ  I am giving 4 stars for the coffee, otherwise ...
5shgJB7a-2_gdnzc0gsOtg  I love Cheesecake Factory period but The Distr...
6tSvz_21BMo3a4GaItwa0g  I love how they have no rice sushi options. Th...
7ibuDLfx8gEhESAufYIySw  Great little sushi bar. Affordable and quaint....
9SU7ZZhaFUJJ6m2k5HKHeg  If I can give 10 stars I will, I don't know wh...
A5Rkh7UymKm0_Rxm9K2PJw  I love Cheesecake Factory period but The Distr...
BLIJ-p5wYuAhw6Pp6mh6mw  Roast beef was amazing that sandwich alone wil...
CVKOPzBVOj3_apFUmZ9ZWw  I got the fish & chips combo, upgraded to the ...
DbEszO3w